In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
# import json
# import PIL

import topovec as tv
# tv.core.configLog() # Enable to see debug output.

In [ ]:
# Settings.
SRC_FOLDER = '../files/' # Location of data files.
IMG_FOLDER = '../tmp/' # Where to put figures.
FIGSIZE = 512 # Image size.
SCALEX = False # FIGSIZE defines height (False) or width (True). 

# Name of datafile.
srcname = 'CCW_800_opposite_mix.npz'

# Load data.
images, system, settings = tv.core.load_lcsim_npz(f"{SRC_FOLDER}/{srcname}")

# `images` is a sequence of vector/director fields.
print(f"Number of images: {len(images)}")

# Select one image
director = images[0]
print(f"Director grid {director.shape}")

# Setup camera.
camera = tv.mgl.Camera()
# Preset 0-2 define axis to look along.
aspectratio = camera.set_predefined_direction(preset=2).adjust_fov(system).adjust_scale(system)
print(f"Aspect ratio {aspectratio:.2f}")

# Define image size
size = (int(FIGSIZE*aspectratio), FIGSIZE) if SCALEX else (FIGSIZE, int(FIGSIZE/aspectratio))

# Select scene to render.
scenecls = tv.mgl.LayerScene # Render single layer.

# Initialize image to render to.
ic = tv.mgl.ImageContainer(size=size,
                           scene=lambda mglctx: scenecls(mglctx=mglctx, system=system, camera=camera),
                          )

# Make director field known to the director. 
ic.upload(director)

# Tune system parameters if needed.
ic.scene.layer = 10

# Render and save to PIL image.
img = ic.save() # PIL image. See PIL documentation for postprocessing.

# Show image in the notebook.
display(img)

# Save image to the file.
# img.save(f"{IMG_FOLDER}/{srcname}.test.jpg")


In [ ]:
# Restrict director field to a sparse grid.
sparse_director, sparse_system = system.thinned(data=director, steps=4)

# Create new container.
ic = tv.mgl.ImageContainer(size=size,
                           scene=lambda mglctx: scenecls(mglctx=mglctx, system=sparse_system, camera=camera),
                          )
ic.upload(sparse_director)

# Render image.
ic.save() 

In [ ]:
# One can reuse existing ImageContainer to render again, if scene and system are not changed.
# Lets try another projection
ic.scene.axis = 0

# Scene can update camera atomatically.
# So if you want to adjust camera manually, do it AFTER setting scene parameters.
# ic.scene.set_camera(camera)

ic.save()
# Image is still square, since aspect ratio was fixed when ImageContained was created.

In [ ]:
# Some operations are very common, so we have helper functions to use them fast.
# This is how we can render single layer as above, but with one line.
ic = tv.mgl.render_layer(sparse_system, layer = 25, axis = 1)
ic.upload(sparse_director)
ic.save()

In [ ]:
# Now render isosurface. Internally it uses FlowScene.
ic = tv.mgl.render_isosurface(system, preset = 0)
ic.upload(director)
ic.save()

In [ ]:
# The same way we can render isolines (IsolinesScene).
ic = tv.mgl.render_isolines(system, preset = 0, nlines=10)
ic.upload(director)
ic.save()

In [ ]:
tv.matplotlib.render_layer(director, layer=10, imagewidth_inch=5);


In [ ]:
# List all avaliable scenes
tv.mgl.print_scenes()

# Prepare scene.
ic = tv.mgl.render_prepare(
    scenecls = 'Astra',
    system = system,
    imgsize = FIGSIZE,
)

# Uploading vector field will initialize properties with default values. 
ic.upload(director)

# List scene properties. The properties are grouped by the render.
ic.scene.print_properties()

# You can access any property by render name and property index or name.
ic.scene['Astra']['Invert colors'] = False

# Visualize the result.
ic.save()

In [ ]:
# Camera settings can be read from the scene.
print(ic.scene.camera)

# The camera settings can be modified.
camera = tv.mgl.Camera.from_angles(theta=2.5, phi=1.0, alpha=0.0, 
                                   scale=30, look_at=[99.5, 99.5, 9.5], perspective=True)
ic.scene.set_camera(camera)

# Let's see the result.
ic.save()

In [ ]:
# Most renders use HSV color scheme to show vector/director orientation.
# Mapping of colors to the Bloch sphere is available from the following helper function.
tv.matplotlib.plot_bloch_sphere(axis=1);

In [ ]:
import matplotlib.pyplot as plt
import taichi as ti
ti.init()

In [ ]:
lambdas = np.linspace(0.380, 1.000, 13)
rgb = tv.ti.render_cp_multi(
    nn=director[:,:,:,0,:], 
    wavelengths=lambdas, 
    ne=tv.ti.ConstantFunction(1.7), 
    no=tv.ti.ConstantFunction(1.5), 
    emission=tv.ti.BlackbodyRadiation(),
    efficiency=tv.ti.SimplifiedRGB(), 
    thickness=settings['L'], 
    polarizer=0., 
    deltafilter=np.pi/2)

plt.imshow(rgb)